#**Data preprocessing & processing**    


###**1. Convert the .faa file into FASTA-format sequences**

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state informa

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Define the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('nferruz/ProtGPT2')
tokenizer = GPT2Tokenizer.from_pretrained('nferruz/ProtGPT2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/655k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/314k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

**To read the file into colab:**

In [3]:
# Authenticate and create the PyDrive client.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1Atd-YHYECzPLF1p4RtJ9x0ProbZg4OA_'}) # replace the id with id of file you want to access
downloaded.GetContentFile('TASmania_hits_seqtk.faa')


In [ ]:
# Split the document into lines. Each line as an element, saved in the list, named text.

with open('TASmania_hits_seqtk.faa', 'r') as fp:
  text = fp.read().split('\n')

In [ ]:
raw_input = []
for t in text:
  if t != '':
    if t[0] != '>':
      raw_input = raw_input + [t]

In [ ]:
raw_input[-3:]

['MLFSAKAEYACIAMLELAARHGDPQPIRLQTMHAEHGIPKRFLVQIMLQLKGAGLVVSVRGATGGYRLAREPQEINLAQIIGVIDRLEPTSTTRQDDRLQQTPTVQTIRSVWREIQSLQQRILTQTTLAELVRRVQDDQSFAYQI',
 'MPERLPPGLLADGPDEMVSGQGRIRPHWRAVIGTVGSLGRNALAERSERLARAMLDEGVASILPGASAAGLWRLDPVPLPLDGAEFEALAAGIAQRARLLDAVLADLYGAQALLAEGVIPPLLVAASPAFLRAARRAMSPPQRPQLHLYAADLLRAPDGRWRVLQDRTAVASGLGQLREN',
 'EHDNAAVFYPGELTLKYDSNISSAAADTLTVIYVPYGVWNNDNWSGTDSFSDYWNDHWSNRECVVLGGRVNASARTITVPFSKTGFGTYAVFNVSREFSDLGPGTNFSWSRNYVLPLWAKGVMEQAGGGNDF']

In [ ]:
max_len=0

for i in raw_input:
  if len(i) > max_len:
    max_len = len(i)

print(max_len)


1499


In [ ]:
min_len=1000000

for i in raw_input:
  if len(i) < min_len:
    min_len = len(i)

print(min_len)

43


In [ ]:
print(raw_input[-1])
print(len(raw_input[-1]))

EHDNAAVFYPGELTLKYDSNISSAAADTLTVIYVPYGVWNNDNWSGTDSFSDYWNDHWSNRECVVLGGRVNASARTITVPFSKTGFGTYAVFNVSREFSDLGPGTNFSWSRNYVLPLWAKGVMEQAGGGNDF
132


**Convert the raw_input into FASTA file format**

In [ ]:
def sequence_conversion(seq):
  count = 0
  converted_seq = ""
  for i in seq:
    if (i != '') & (i != '>'):
      converted_seq = converted_seq + i
      count = count + 1
      if count % 60 == 0:
        converted_seq = converted_seq + '\n'
  return converted_seq                 #return "<|endoftext|>" + '\n'+ converted_seq + "<|endoftext|>"

**To test the function**

In [ ]:
a3 = 'LVHGFGASIGHWRHNIPVLAAAGYRVYALDLLGFGGSAKPALAYSLELWAELLADFWRQHVRQPAVFIGNSIGALLSLLMAARYPQLVAGAVLLNCAGGLNHRSHELSPLARLFMATFTALVASPVTGPLLFDWVRQRQRIRATLKQVYCNPAAITDELVEILYAPSCEPGAQQVFAAVLTAPPGPTPEALLPQVRCPLLVIWGERDPWTPIQRGRGFQNHVQGIDYQFFPLPDTGHCPHDERPEIVNPLILSWLPKAFATA'

sequence_conversion(a3)

'LVHGFGASIGHWRHNIPVLAAAGYRVYALDLLGFGGSAKPALAYSLELWAELLADFWRQH\nVRQPAVFIGNSIGALLSLLMAARYPQLVAGAVLLNCAGGLNHRSHELSPLARLFMATFTA\nLVASPVTGPLLFDWVRQRQRIRATLKQVYCNPAAITDELVEILYAPSCEPGAQQVFAAVL\nTAPPGPTPEALLPQVRCPLLVIWGERDPWTPIQRGRGFQNHVQGIDYQFFPLPDTGHCPH\nDERPEIVNPLILSWLPKAFATA'

In [ ]:
print(sequence_conversion(a3))

LVHGFGASIGHWRHNIPVLAAAGYRVYALDLLGFGGSAKPALAYSLELWAELLADFWRQH
VRQPAVFIGNSIGALLSLLMAARYPQLVAGAVLLNCAGGLNHRSHELSPLARLFMATFTA
LVASPVTGPLLFDWVRQRQRIRATLKQVYCNPAAITDELVEILYAPSCEPGAQQVFAAVL
TAPPGPTPEALLPQVRCPLLVIWGERDPWTPIQRGRGFQNHVQGIDYQFFPLPDTGHCPH
DERPEIVNPLILSWLPKAFATA


**Create the FASTA-format file, named converted_sequences.**

In [ ]:
converted_sequences = []
for i in raw_input:
  converted_sequences = converted_sequences + [sequence_conversion(i)]

In [ ]:
converted_sequences

['MASPELEVLGITTVAGNVSVEKTSRNARQICELAGCPQMAVYAGCPRPLLRPLQTAEEVH\nGKSGIDGANLPEPQMPLGSLHAVQYLIETLMAAQEPVTLALLGPMTNLAVALVQQPRIVE\nRIRRLVFMGGSAFEGNTTPAAEFNIFTDPHAAQIVLSAGIPEVVMLGLNVTQQVLSTPER\nIERIRALGTRVGQTVADMLAFYGKFDIRRYGLPGGPLHDPCVVAYLLQPQLFELKPCYVE\nVETASPLNLGRTVVDRWGLSGRPANVQVAFGVDAEEFYRLLTERLGRYR',
 'MLRQLRPVDKSGKMGSLLWPLLFGASLALSPAIEPAWGQETSQNHGTASNVPLLSRLDAL\nEELRQALSLPTASIQTEAVTLDGRRLFLVAAPQGDPGSLPPVSQRARSIPNSAIRVVENL\nSNGWSRVDLGIVIAYENDLEQAIRITEQVALQMYREPAWRDKIREPPEMHGVDDLGERGI\nRLRIWIKVQPLQQWRVTREYRRRLKHAFDQAGIQIPFPQQALWFRSPLEMRIQGLSPEET\nHRLLHPMESRLSNPDPISEERAGQLPP',
 'LVHGFGASIGHWRHNIPVLAAAGYRVYALDLLGFGGSAKPALAYSLELWAELLADFWRQH\nVRQPAVFIGNSIGALLSLLMAARYPQLVAGAVLLNCAGGLNHRSHELSPLARLFMATFTA\nLVASPVTGPLLFDWVRQRQRIRATLKQVYCNPAAITDELVEILYAPSCEPGAQQVFAAVL\nTAPPGPTPEALLPQVRCPLLVIWGERDPWTPIQRGRGFQNHVQGIDYQFFPLPDTGHCPH\nDERPEIVNPLILSWLPKAFATA',
 'MRGFSLWQSWKAAAVIASWGSLLLVADPVWAQLSDISTYWGAQYIAGLNERQIIGGFPDG\nTFRPNESITRAQFAVIVTKAFGLDTNVPVRPFADPIPSWAAPSIGAAAAAGLVSGFPDGT\nFRPNDVLTRAQAITVLT

In [ ]:
len(converted_sequences)

440

###**2. Use the sequences to build Dataset**

**To see the Encode and Decode:**

In [ ]:
print(tokenizer.encode(converted_sequences[0]))

[20797, 598, 545, 321, 5599, 1333, 286, 1186, 12928, 454, 2320, 273, 5124, 1123, 1419, 549, 16148, 1015, 447, 530, 392, 199, 561, 277, 720, 620, 39407, 24891, 263, 1507, 1589, 3019, 852, 345, 11109, 1770, 48, 22820, 3760, 1325, 48, 4931, 199, 1427, 29966, 16556, 9305, 1612, 42256, 427, 12890, 576, 38894, 625, 273, 1203, 274, 12735, 1411, 341, 6506, 6575, 199, 49917, 504, 7753, 2838, 6978, 10741, 1872, 1043, 21203, 4569, 2881, 1937, 355, 6915, 283, 1264, 374, 660, 286, 199, 286, 927, 279, 743, 11818, 467, 2228, 15312, 29017, 8237, 294, 530, 446, 623, 340, 587, 19700]


If no '\n', the output will be different. Like this:  
```
[20797, 598, 545, 321, 5599, 1333, 286, 1186, 12928, 454, 2320, 273, 5124, 1123, 1419, 549, 16148, 1015, 447, 530, 1546, 982, 720, 620, 39407, 24891, 263, 1507, 1589, 3019, 852, 345, 11109, 1770, 48, 22820, 3760, 1325, 48, 4931, 1427, 29966, 16556, 9305, 1612, 42256, 427, 12890, 576, 38894, 625, 273, 1203, 274, 12735, 1411, 341, 6506, 298, 1133, 1427, 504, 7753, 2838, 6978, 10741, 1872, 1043, 21203, 4569, 2881, 1937, 355, 6915, 283, 1264, 374, 660, 3957, 927, 279, 743, 11818, 467, 2228, 15312, 29017, 8237, 294, 530, 446, 623, 340, 587, 19700]
```

**Tokenize the sequences:**

In [ ]:
ids_input = []
for i in converted_sequences:
  ids_input = ids_input + [(lambda x: tokenizer.encode(x))(i)]

In [ ]:
print(ids_input)

[[20797, 598, 545, 321, 5599, 1333, 286, 1186, 12928, 454, 2320, 273, 5124, 1123, 1419, 549, 16148, 1015, 447, 530, 392, 199, 561, 277, 720, 620, 39407, 24891, 263, 1507, 1589, 3019, 852, 345, 11109, 1770, 48, 22820, 3760, 1325, 48, 4931, 199, 1427, 29966, 16556, 9305, 1612, 42256, 427, 12890, 576, 38894, 625, 273, 1203, 274, 12735, 1411, 341, 6506, 6575, 199, 49917, 504, 7753, 2838, 6978, 10741, 1872, 1043, 21203, 4569, 2881, 1937, 355, 6915, 283, 1264, 374, 660, 286, 199, 286, 927, 279, 743, 11818, 467, 2228, 15312, 29017, 8237, 294, 530, 446, 623, 340, 587, 19700], [339, 970, 17566, 982, 43, 1661, 257, 712, 4537, 503, 5282, 13228, 2163, 345, 37124, 366, 927, 1564, 570, 269, 735, 199, 37, 5892, 2593, 17829, 387, 9900, 4729, 782, 288, 550, 48, 1897, 296, 4260, 26745, 933, 39422, 4036, 702, 287, 199, 23969, 552, 436, 11958, 11797, 448, 15731, 278, 1310, 492, 603, 585, 1752, 12703, 41745, 2316, 298, 3381, 3870, 428, 7755, 199, 25309, 3111, 362, 23255, 595, 9768, 1676, 24025, 31031, 1467

**Build the attention_mask list**

In [ ]:
def get_attention_mask(x: list):
  j = []
  for i in range(len(x)):
    j = j + [1]
  return j

In [ ]:
attention_mask_input = []
for i in ids_input:
  attention_mask_input = attention_mask_input + [(lambda x: get_attention_mask(x))(i)]

In [ ]:
print(attention_mask_input)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

**Build a Dataframe**

In [ ]:
import pandas as pd

df=pd.DataFrame()
df['attention_mask'] = attention_mask_input
df['input_ids'] = ids_input
df

,attention_mask,input_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[20797, 598, 545, 321, 5599, 1333, 286, 1186, ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[339, 970, 17566, 982, 43, 1661, 257, 712, 453..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[44, 1546, 329, 7067, 1030, 440, 1502, 260, 18..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2038, 960, 3826, 25238, 271, 839, 2717, 442, ..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[27299, 683, 378, 8870, 19247, 519, 13923, 430..."
...,...,...
435,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[8412, 732, 510, 257, 3303, 1516, 1747, 259, 6..."
436,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[36, 673, 815, 388, 1649, 586, 13974, 282, 218..."
437,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19072, 5342, 4116, 18817, 262, 683, 29032, 40..."
438,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2088, 269, 854, 257, 631, 4356, 12868, 348, 1..."


**Convert the Dataframe into Dataset and split it into train dataset and validation dataset**

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 440
})

In [ ]:
raw_datasets = dataset.train_test_split(test_size=0.1)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 396
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 44
    })
})

In [ ]:
# Rename the default "test" split to "validation"
raw_datasets["validation"] = raw_datasets.pop("test")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 396
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 44
    })
})

**Convert the DatasetDict into JSON format, then Streaming.**

In [ ]:
for split, dataset in raw_datasets.items():
  dataset.to_json(f"stream-dataset-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

###**3. Create a dataset "Lifan-Z/tox-antitox-proteins" in Hugging Face**  


Login to Hugging Face account, then create a dataset "**Lifan-Z/tox-antitox-proteins**" and upload the files **stream-dataset-train.jsonl** and **stream-dataset-validation.jsonl** to the dataset directory.

**Test the dataset**

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader

remote_train = load_dataset("Lifan-Z/tox-antitox-proteins", split="train")
remote_validation = load_dataset("Lifan-Z/tox-antitox-proteins", split="validation")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
remote_train

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 396
})

In [ ]:
remote_validation

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 44
})

In [ ]:
print(remote_validation[0])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'input_ids': [2038, 960, 3826, 25238, 271, 839, 2717, 442, 718, 14540, 580, 417, 30073, 41044, 559, 472, 49661, 3439, 8868, 199, 28585, 1793, 321, 1343, 1160, 1107, 1064, 654, 407, 10765, 507, 281, 23959, 2211, 318, 299, 789, 472, 682, 421, 7089, 199, 2356, 391, 9164, 6281, 793, 16871, 1536, 623, 414, 296, 1370, 30130, 1755, 620, 333, 499, 14217, 924, 271, 1068, 199, 2811, 2041, 21067, 1758, 3754, 43571, 430, 425, 716, 9168, 1509, 777, 356, 3623, 1088, 296, 13664, 2974, 54, 199, 356, 928, 6545, 370, 7358, 279, 3717, 833, 8686, 13023, 692, 7906, 316, 1254, 267, 2045, 3533, 27353, 7376, 199, 1088, 1556, 6264, 345, 18687, 2

**The dataset works well.**